# March Madness Prediction

## Overview

###

### Goal
Submissions are based on the Brier Score, the goal will be to minimize the brier score between the predicted probabilities and the actual game outcomes. The Brier score measures the accuracy of probablistic predition, in this case the mean square error. 

The brier score can be thought of as a cost function that measures the average squared difference between the predicted probabilities and the actual outcomes.

$$
Brier = \frac{1}{N} \sum_{i=1}^{N} (p_i - o_i)^2
$$

where $p_i$ is the predicted probability of the event and $o_i$ is the actual outcome. The Brier score can span across all items in a set of N predictions.

Therefore, minimizing the Brier score will result in a more accurate prediction.


In [2]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import xgboost as xgb
import sklearn as sk
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, mean_absolute_error, brier_score_loss

# Style
plt.style.use("dark_background")
px.defaults.template = 'plotly_dark'

## Load Data

Set up a data dictionary that will store the data for each file, this will make it easier to access data from the csvs. Not all files are used in the prediction process, but they are included for completeness.


In [3]:
# Load Data
path = 'data/**'
data = {p.split('/')[-1].split('.')[0] : pd.read_csv(p, encoding='latin-1') for p in glob.glob(path)}

# Create Teams Dataframe
teams = pd.concat([data['MTeams'], data['WTeams']])
teams_spelling = pd.concat([data['MTeamSpellings'], data['WTeamSpellings']])
teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams_spelling.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])

# Create Season Dataframes and S/T Flag
season_compact_results = pd.concat([data['MRegularSeasonCompactResults'], data['WRegularSeasonCompactResults']]).assign(ST='S')
season_detailed_results = pd.concat([data['MRegularSeasonDetailedResults'], data['WRegularSeasonDetailedResults']]).assign(ST='S')
tourney_compact_results = pd.concat([data['MNCAATourneyCompactResults'], data['WNCAATourneyCompactResults']]).assign(ST='T')
tourney_detailed_results = pd.concat([data['MNCAATourneyDetailedResults'], data['WNCAATourneyDetailedResults']]).assign(ST='T')

# Create Tourney Dataframes
lots = pd.concat([data['MNCAATourneySlots'], data['WNCAATourneySlots']])
seeds = pd.concat([data['MNCAATourneySeeds'], data['WNCAATourneySeeds']])
seeds['SeedValue'] = seeds['Seed'].str.extract(r'(\d+)').astype(int)
seeds_dict = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
game_cities = pd.concat([data['MGameCities'], data['WGameCities']])
seasons = pd.concat([data['MSeasons'], data['WSeasons']])
cities = data['Cities']

# Create Sample Submission Dataframe
sub = data['SampleSubmissionStage1']
del data

# Seeds Dictionary
seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}

## Feature Engineering


In [4]:
# Create Detailed Results Dataframe
all_detailed_results = pd.concat([season_detailed_results, tourney_detailed_results])
all_detailed_results.reset_index(drop=True, inplace=True)
all_detailed_results['WLoc'] = all_detailed_results['WLoc'].map({'A': 1, 'H': 2, 'N': 3})

# Add additional features to detailed results
all_detailed_results['ID'] = all_detailed_results.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
all_detailed_results['IDTeams'] = all_detailed_results.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
all_detailed_results['Team1'] = all_detailed_results.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
all_detailed_results['Team2'] = all_detailed_results.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
all_detailed_results['IDTeam1'] = all_detailed_results.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
all_detailed_results['IDTeam2'] = all_detailed_results.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)

all_detailed_results['Team1Seed'] = all_detailed_results['IDTeam1'].map(seeds).fillna(0)
all_detailed_results['Team2Seed'] = all_detailed_results['IDTeam2'].map(seeds).fillna(0)

all_detailed_results['ScoreDiff'] = all_detailed_results['WScore'] - all_detailed_results['LScore']
all_detailed_results['Pred'] = all_detailed_results.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
all_detailed_results['ScoreDiffNorm'] = all_detailed_results.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
all_detailed_results['SeedDiff'] = all_detailed_results['Team1Seed'] - all_detailed_results['Team2Seed'] 
all_detailed_results = all_detailed_results.fillna(-1)

# Add derived features to detaifled results
all_detailed_results['ScoreDiff'] = all_detailed_results['WScore'] - all_detailed_results['LScore']
all_detailed_results['HomeAdvantage'] = (all_detailed_results['WLoc'] == 2).astype(int)

# Calculate shooting percentages (handling division by zero)
all_detailed_results['WFGPct'] = np.where(all_detailed_results['WFGA'] > 0, 
                                        all_detailed_results['WFGM'] / all_detailed_results['WFGA'], 0)
all_detailed_results['WFG3Pct'] = np.where(all_detailed_results['WFGA3'] > 0, 
                                        all_detailed_results['WFGM3'] / all_detailed_results['WFGA3'], 0)
all_detailed_results['WFTPct'] = np.where(all_detailed_results['WFTA'] > 0, 
                                        all_detailed_results['WFTM'] / all_detailed_results['WFTA'], 0)
all_detailed_results['LFGPct'] = np.where(all_detailed_results['LFGA'] > 0, 
                                        all_detailed_results['LFGM'] / all_detailed_results['LFGA'], 0)
all_detailed_results['LFG3Pct'] = np.where(all_detailed_results['LFGA3'] > 0, 
                                        all_detailed_results['LFGM3'] / all_detailed_results['LFGA3'], 0)
all_detailed_results['LFTPct'] = np.where(all_detailed_results['LFTA'] > 0, 
                                        all_detailed_results['LFTM'] / all_detailed_results['LFTA'], 0)

# Add statistical differences
all_detailed_results['ReboundDiff'] = (all_detailed_results['WOR'] + all_detailed_results['WDR']) - \
                                    (all_detailed_results['LOR'] + all_detailed_results['LDR'])
all_detailed_results['AssistDiff'] = all_detailed_results['WAst'] - all_detailed_results['LAst']
all_detailed_results['TurnoverDiff'] = all_detailed_results['WTO'] - all_detailed_results['LTO']
all_detailed_results['StealDiff'] = all_detailed_results['WStl'] - all_detailed_results['LStl']
all_detailed_results['BlockDiff'] = all_detailed_results['WBlk'] - all_detailed_results['LBlk']
all_detailed_results['FoulDiff'] = all_detailed_results['WPF'] - all_detailed_results['LPF']

In [5]:
c_score_col = ['NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',
 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
 'LBlk', 'LPF']
c_score_agg = ['sum', 'mean', 'median', 'max', 'min', 'std', 'skew', 'nunique']
gb = all_detailed_results.groupby(by=['IDTeams']).agg({k: c_score_agg for k in c_score_col}).reset_index()
gb.columns = [''.join(c) + '_c_score' for c in gb.columns]

sub['WLoc'] = 3
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 
sub = sub.fillna(-1)

games = pd.merge(all_detailed_results, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
sub = pd.merge(sub, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')

col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm', 'WLoc'] + c_score_col]

In [6]:
imputer = SimpleImputer(strategy='mean')  
scaler = StandardScaler()

X = games[col].fillna(-1)
missing_cols = set(col) - set(sub.columns)
for c in missing_cols:
    sub[c] = 0

X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)

# Main prediction model
model = xgb.XGBRegressor(
    n_estimators=5000,         # Number of boosting rounds
    learning_rate=0.05,       # Smaller learning rate for better generalization
    max_depth=6,              # Control model complexity
    min_child_weight=3,       # Helps prevent overfitting
    subsample=0.8,            # Use 80% of data for each tree
    colsample_bytree=0.8,     # Use 80% of features for each tree
    objective='binary:logistic',  # Binary classification with probability output
    random_state=42,
    n_jobs=-1                 # Use all CPU cores
)

# Train the main model
print("\nTraining main model...")
model.fit(X_scaled, games['Pred'])

pred = model.predict(X_scaled).clip(0.001, 0.999)


print(f'Log Loss: {log_loss(games["Pred"], pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(games["Pred"], pred)}')
print(f'Brier Score: {brier_score_loss(games["Pred"], pred)}')

cv_scores = cross_val_score(model, X_scaled, games['Pred'], cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated MSE: {-cv_scores.mean()}')



Training main model...
Log Loss: 0.2836254754686792
Mean Absolute Error: 0.229211302581863
Brier Score: 0.07603705889507632
Cross-validated MSE: 0.21865491983811386


In [ ]:
# causing crash perhaps?
sub_X = sub[col].fillna(-1)
# Add print statements to debug
print("Shape of sub_X:", sub_X.shape)
print("Creating imputed data...")

# todo: investigate why imputer transform on sub_X is causing a crash
sub_X_imputed = imputer.transform(sub_X)
print("Scaling data...")
sub_X_scaled = scaler.transform(sub_X_imputed)

print("Making predictions...")
# Store predictions in a variable first
predictions = model.predict(sub_X_scaled).clip(0.001, 0.999)
print("Prediction shape:", predictions.shape)

# Assign to dataframe
sub['Pred'] = predictions
print("Saving to CSV...")
sub[['ID', 'Pred']].to_csv('submission.csv', index=False)
print("Done!")

Shape of sub_X: (507108, 234)
Creating imputed data...
